In [ ]:
# Perry H. Zhang perry.h.zhang.24@dartmouth.edu
# The program used to scrape the data that we use for GPT2
# NOTE: Because of the nature of the requests sent by lyricsgenius, often times than not the requests
# will time out, EVEN IF stop times are introduced. In my case, having a stop time of 150 seconds
# for a hundred successful song queries alone couldn't even solve this issue.

In [1]:
#importing the libraries
import requests
import pprint
import time
import random


In [2]:
# number of total successful song requests

n_succ_ideal = 5000

In [3]:
############### Spotify ###############

In [4]:
# cell to specify some necessary things for querying

# link https://developer.spotify.com/dashboard/applications/d2c911ec6aef475a8ece33456a3161ea

CLIENT_ID = 'd2c911ec6aef475a8ece33456a3161ea'
CLIENT_SECRET = '2f09f748238949219359c6d64d320fce'


AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

#sanity check that we have the oauth token working
pprint.pprint(access_token)

# attach the oauth token to 
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

'BQAaGj0b8H8qy1VBK77Fhm546MasLqGkNOQpo_A0KPG0LYdthPfUY8tNsdZMpuMwv1KGWt48ssfrSmsAomg'


In [5]:
# step 1. get the genres of spotify songs https://developer.spotify.com/console/get-available-genre-seeds/

spotify_genres = []

# actual GET request with proper header
genre_r = requests.get(BASE_URL + 'recommendations/available-genre-seeds', headers=headers)


In [6]:
genre_r = genre_r.json()

#set the spotify genres to that of which we retrieved from the response
spotify_genres = genre_r["genres"]

In [7]:
# step 2. get the playlists based on the category
# endpoint to get the songs https://developer.spotify.com/documentation/web-api/reference/#endpoint-get-a-categories-playlists

# spotify genre playlists
spotify_genre_playlists = {}

# define a function that we use to get the playlists
def get_playlists_from_genre(genre):
    limit_playlists = 35

    # actual GET request with proper header
    # this is just a test at the moment
    playlist_r = requests.get(BASE_URL + 'browse/categories/' + genre + '/playlists?offset=0&limit=' + str(limit_playlists), headers=headers)
    playlist_r = playlist_r.json()
    
    if 'error' not in playlist_r:
        return playlist_r['playlists']['items']
    else:
        return None
    

In [8]:
for genre in spotify_genres:
    genre_playlist_info = get_playlists_from_genre(genre)
    if genre_playlist_info != None:
        spotify_genre_playlists[genre] = genre_playlist_info

#sanity check
pprint.pprint(spotify_genre_playlists)

{'ambient': [{'collaborative': False,
              'description': 'Music to induce calm and a space to think. '
                             'Cover: Sufjan Stevens',
              'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DX9c7yCloFHHL'},
              'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX9c7yCloFHHL',
              'id': '37i9dQZF1DX9c7yCloFHHL',
              'images': [{'height': None,
                          'url': 'https://i.scdn.co/image/ab67706f00000003b5fbb0f8fa7ede291e6b8bb6',
                          'width': None}],
              'name': 'Ambiente',
              'owner': {'display_name': 'Spotify',
                        'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
                        'href': 'https://api.spotify.com/v1/users/spotify',
                        'id': 'spotify',
                        'type': 'user',
                        'uri': 'spotify:user:spotify'},
              'prim

                      'uri': 'spotify:user:spotify'},
            'primary_color': None,
            'public': None,
            'snapshot_id': 'MTU5NzI2NjE1MywwMDAwMDAwZTAwMDAwMTczZTQ3YjUxMjYwMDAwMDE3MGE0ZWUwYmZm',
            'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX7KrTMVQnM02/tracks',
                       'total': 30},
            'type': 'playlist',
            'uri': 'spotify:playlist:37i9dQZF1DX7KrTMVQnM02'},
           {'collaborative': False,
            'description': 'Relax to the calming sounds of choral music.',
            'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DX9WxEZbyU6MA'},
            'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX9WxEZbyU6MA',
            'id': '37i9dQZF1DX9WxEZbyU6MA',
            'images': [{'height': None,
                        'url': 'https://i.scdn.co/image/ab67706f000000030889aae034b2c42be9455620',
                        'width': None}],
            'name': 'Peaceful C

                               'piano works by Chopin.',
                'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DWZYpU3rwrcAz'},
                'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DWZYpU3rwrcAz',
                'id': '37i9dQZF1DWZYpU3rwrcAz',
                'images': [{'height': None,
                            'url': 'https://i.scdn.co/image/ab67706f000000038e00671219dc7251b38b8b4d',
                            'width': None}],
                'name': 'Relaxing Chopin',
                'owner': {'display_name': 'Spotify',
                          'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
                          'href': 'https://api.spotify.com/v1/users/spotify',
                          'id': 'spotify',
                          'type': 'user',
                          'uri': 'spotify:user:spotify'},
                'primary_color': None,
                'public': None,
                'snapshot

              'public': None,
              'snapshot_id': 'MTYyMzE0MjYzMSwwMDAwMDAwMGQ0MWQ4Y2Q5OGYwMGIyMDRlOTgwMDk5OGVjZjg0Mjdl',
              'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX0vVAWnG6fKT/tracks',
                         'total': 70},
              'type': 'playlist',
              'uri': 'spotify:playlist:37i9dQZF1DX0vVAWnG6fKT'}],
 'funk': [{'collaborative': False,
           'description': 'New funky songs with a classic funky sound.',
           'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DX4WgZiuR77Ef'},
           'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX4WgZiuR77Ef',
           'id': '37i9dQZF1DX4WgZiuR77Ef',
           'images': [{'height': None,
                       'url': 'https://i.scdn.co/image/ab67706f000000031ef7bd677c0a00cb77f1b6e3',
                       'width': None}],
           'name': 'All Funked Up',
           'owner': {'display_name': 'Spotify',
                     'external_u

               'primary_color': None,
               'public': None,
               'snapshot_id': 'MTU3NzI4NzM2MSwwMDAwMDAxODAwMDAwMTZmM2RhNzI0YTUwMDAwMDE2ZTkxY2ZmODcx',
               'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DWWTuLtC3XxmA/tracks',
                          'total': 130},
               'type': 'playlist',
               'uri': 'spotify:playlist:37i9dQZF1DWWTuLtC3XxmA'},
              {'collaborative': False,
               'description': 'From joyful jubillance to hushed harmonies – '
                              'this is some of the best in A Cappella '
                              'Christmas.',
               'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DWTZ37yWkEort'},
               'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DWTZ37yWkEort',
               'id': '37i9dQZF1DWTZ37yWkEort',
               'images': [{'height': None,
                           'url': 'https://i.scdn.co/image/ab67706f00

...

            'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DWXHwQpcoF2cC/tracks',
                       'total': 140},
            'type': 'playlist',
            'uri': 'spotify:playlist:37i9dQZF1DWXHwQpcoF2cC'},
           {'collaborative': False,
            'description': 'Whether you call it technical, progressive or math '
                           'metal, it all djents. Cover: Erra',
            'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DX6GRSnGELn7L'},
            'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX6GRSnGELn7L',
            'id': '37i9dQZF1DX6GRSnGELn7L',
            'images': [{'height': None,
                        'url': 'https://i.scdn.co/image/ab67706f0000000387018c1e55cf2db4bb21df20',
                        'width': None}],
            'name': 'Got Djent?',
            'owner': {'display_name': 'Spotify',
                      'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},


           'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX2UXfvEIZvDK/tracks',
                      'total': 60},
           'type': 'playlist',
           'uri': 'spotify:playlist:37i9dQZF1DX2UXfvEIZvDK'},
          {'collaborative': False,
           'description': 'The biggest rock songs of the last decade. Cover: '
                          'Arctic Monkeys',
           'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DX99DRG9N39X3'},
           'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX99DRG9N39X3',
           'id': '37i9dQZF1DX99DRG9N39X3',
           'images': [{'height': None,
                       'url': 'https://i.scdn.co/image/ab67706f0000000382f0c8b77359661093a343df',
                       'width': None}],
           'name': '10s Rock Anthems',
           'owner': {'display_name': 'Spotify',
                     'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
                     'href': 'h

             {'collaborative': False,
              'description': '¡Que viva el amor!',
              'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DWUoGbRYcteyC'},
              'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DWUoGbRYcteyC',
              'id': '37i9dQZF1DWUoGbRYcteyC',
              'images': [{'height': None,
                          'url': 'https://i.scdn.co/image/ab67706f00000003b36611c91dfbfaa0176ad133',
                          'width': None}],
              'name': 'Amor Amor',
              'owner': {'display_name': 'Spotify',
                        'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
                        'href': 'https://api.spotify.com/v1/users/spotify',
                        'id': 'spotify',
                        'type': 'user',
                        'uri': 'spotify:user:spotify'},
              'primary_color': None,
              'public': None,
              'snapshot_id': 

             'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DWSf2RDTDayIx',
             'id': '37i9dQZF1DWSf2RDTDayIx',
             'images': [{'height': None,
                         'url': 'https://i.scdn.co/image/ab67706f00000003fe0099a8dcd3054706ffc92f',
                         'width': None}],
             'name': 'Happy Beats',
             'owner': {'display_name': 'Spotify',
                       'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
                       'href': 'https://api.spotify.com/v1/users/spotify',
                       'id': 'spotify',
                       'type': 'user',
                       'uri': 'spotify:user:spotify'},
             'primary_color': None,
             'public': None,
             'snapshot_id': 'MTYyMzE0MjY2NSwwMDAwMDAwMGQ0MWQ4Y2Q5OGYwMGIyMDRlOTgwMDk5OGVjZjg0Mjdl',
             'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DWSf2RDTDayIx/tracks',
                        'total':

             'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DWVcbzTgVpNRm'},
             'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DWVcbzTgVpNRm',
             'id': '37i9dQZF1DWVcbzTgVpNRm',
             'images': [{'height': None,
                         'url': 'https://i.scdn.co/image/ab67706f0000000361761ca71ae4a58e64bda477',
                         'width': None}],
             'name': 'Latin Party Anthems',
             'owner': {'display_name': 'Spotify',
                       'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
                       'href': 'https://api.spotify.com/v1/users/spotify',
                       'id': 'spotify',
                       'type': 'user',
                       'uri': 'spotify:user:spotify'},
             'primary_color': None,
             'public': None,
             'snapshot_id': 'MTYyMzE0MjY0MSwwMDAwMDAwMGQ0MWQ4Y2Q5OGYwMGIyMDRlOTgwMDk5OGVjZjg0Mjdl',
             'tracks': 

In [9]:
len(spotify_genre_playlists)

18

In [10]:
# step 3. get songs within each playlist
# sample endpoint "https://api.spotify.com/v1/playlists/37i9dQZF1DWUrPBdYfoJvz/tracks?market=US&limit=10&offset=5"

# define a function that gets the songs from a playlist
def get_songs_from_playlist(playlist_id):
    limit_songs = 25
    
    # actual GET request with proper header
    # this is just a test at the moment
    
    playlist_song_r = requests.get(BASE_URL + "playlists/" + playlist_id + "/tracks?limit=" + str(limit_songs), headers=headers)
    playlist_song_r = playlist_song_r.json()
    
    if 'error' not in playlist_song_r:
        return playlist_song_r["items"]
    else:
        return None    

In [11]:
#sanity check
get_songs_from_playlist("37i9dQZF1DWYV2Gh2QglGo")

[{'added_at': '2021-06-07T12:21:02Z',
  'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/'},
   'href': 'https://api.spotify.com/v1/users/',
   'id': '',
   'type': 'user',
   'uri': 'spotify:user:'},
  'is_local': False,
  'primary_color': None,
  'track': {'album': {'album_type': 'album',
    'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/1w9VjfHGE5qPLJQy3dKups'},
      'href': 'https://api.spotify.com/v1/artists/1w9VjfHGE5qPLJQy3dKups',
      'id': '1w9VjfHGE5qPLJQy3dKups',
      'name': 'Gord Bamford',
      'type': 'artist',
      'uri': 'spotify:artist:1w9VjfHGE5qPLJQy3dKups'}],
    'available_markets': ['AD',
     'AE',
     'AG',
     'AL',
     'AM',
     'AO',
     'AR',
     'AT',
     'AZ',
     'BA',
     'BB',
     'BD',
     'BE',
     'BF',
     'BG',
     'BH',
     'BI',
     'BJ',
     'BN',
     'BO',
     'BR',
     'BS',
     'BT',
     'BW',
     'BY',
     'BZ',
     'CA',
     'CH',
     'CI',
     'CL',
     

In [12]:
songs_information = []

# for a given genre
for curr_genre_key in spotify_genre_playlists.keys():
    # get the genre's playlists from the "spotify_genre_playlists"
    curr_genre_playlists = spotify_genre_playlists[curr_genre_key]
    
    # for each playlist in the current genre's list of playlists
    for playlist_info in curr_genre_playlists:
        #get the current playlist's id
        curr_playlist_id = playlist_info['id']
        
        #get the list of songs in the playlist
        curr_playlist_songs = get_songs_from_playlist(curr_playlist_id)
        
        # for each song's info in the current playlist
        for song_info in curr_playlist_songs:
            
            # if there exists a track
            if song_info["track"] != None:
                #take the name of the song and the name of the first artist in the song and store
                songs_information.append({"song_name" : song_info["track"]["name"], "song_first_artist" : song_info["track"]["artists"][0]['name']})


In [13]:
# sanity check

pprint.pprint(songs_information)

[{'song_first_artist': 'Sufjan Stevens', 'song_name': 'Lamentation I'},
 {'song_first_artist': 'Martin Kohlstedt', 'song_name': 'XEO'},
 {'song_first_artist': 'Loscil', 'song_name': 'Vespera'},
 {'song_first_artist': 'Ólafur Arnalds', 'song_name': 'Loom (feat. Bonobo)'},
 {'song_first_artist': 'Alessandro Cortini', 'song_name': 'VERDE'},
 {'song_first_artist': 'Christian Löffler', 'song_name': 'Pastoral'},
 {'song_first_artist': 'Grandbrothers', 'song_name': 'What We See'},
 {'song_first_artist': 'Kyson', 'song_name': 'After the Rain - Piano Solo'},
 {'song_first_artist': 'Slow Meadow', 'song_name': 'By the Ash Tree'},
 {'song_first_artist': 'Theo Alexander',
  'song_name': 'Accidental Enlightenment'},
 {'song_first_artist': 'Phoria', 'song_name': 'Nouveau-Né (Re-Imagined)'},
 {'song_first_artist': 'Martin Kohlstedt', 'song_name': 'JUL'},
 {'song_first_artist': 'Ólafur Arnalds', 'song_name': 'Still / Sound'},
 {'song_first_artist': 'Robert Ekson', 'song_name': 'Red Room'},
 {'song_firs

 {'song_first_artist': 'James Heather', 'song_name': 'Glimmer'},
 {'song_first_artist': 'Neil Cowley', 'song_name': 'Don’t Walk Alone'},
 {'song_first_artist': 'Skalpel', 'song_name': 'Escape - Hania Rani Remix'},
 {'song_first_artist': 'Henrik Lindstrand',
  'song_name': 'Søndermarken - Video Edit'},
 {'song_first_artist': 'Elskavon', 'song_name': 'Itasca'},
 {'song_first_artist': 'Jon Hopkins', 'song_name': 'Modern Driveway'},
 {'song_first_artist': 'Balmorhea', 'song_name': 'The Myth'},
 {'song_first_artist': 'Olivia Belli', 'song_name': 'Still Blue'},
 {'song_first_artist': 'Nils Frahm', 'song_name': 'No Step on Wing'},
 {'song_first_artist': 'Vanbur',
  'song_name': 'In Cold Light (Alexandra Hamilton-Ayres Rework)'},
 {'song_first_artist': 'Ólafur Arnalds', 'song_name': 'Back To The Sky'},
 {'song_first_artist': 'Tony Anderson', 'song_name': 'Retour'},
 {'song_first_artist': 'Nils Frahm', 'song_name': 'Forever Changeless'},
 {'song_first_artist': 'Tom Ashbrook', 'song_name': 'Pete

  'song_name': 'My Journey - Film Version for "The Father" / David Menke '
               'Remix'},
 {'song_first_artist': 'Stephanie Economou', 'song_name': 'Illumination'},
 {'song_first_artist': 'Volker Bertelmann', 'song_name': 'I Was in the Fire'},
 {'song_first_artist': 'Dave Rowntree', 'song_name': 'The Mole'},
 {'song_first_artist': 'Kris Bowers',
  'song_name': 'Strange (feat. Hillary Smith)'},
 {'song_first_artist': 'Cliff Martinez', 'song_name': 'My Life is Out There'},
 {'song_first_artist': 'Daniel Pemberton',
  'song_name': 'Enola Holmes (Wild Child)'},
 {'song_first_artist': 'Mark Isham', 'song_name': 'The Nevers'},
 {'song_first_artist': 'Ludwig Goransson', 'song_name': 'Mando Is Back'},
 {'song_first_artist': 'Kevin Penkin',
  'song_name': 'Penrose Steps, A.I. Bloom - From "Eden" Soundtrack'},
 {'song_first_artist': 'Bobby Krlic', 'song_name': 'Helios'},
 {'song_first_artist': 'Patrick Jonsson',
  'song_name': 'No Ball Games (Original Motion Picture Soundtrack)'},
 {'s

 {'song_first_artist': 'Conway Twitty',
  'song_name': "I'd Love To Lay You Down"},
 {'song_first_artist': 'David Frizzell',
  'song_name': "You're The Reason God Made Oklahoma"},
 {'song_first_artist': 'Don Gibson', 'song_name': 'Sea of Heartbreak'},
 {'song_first_artist': 'Roger Miller', 'song_name': 'King Of The Road'},
 {'song_first_artist': 'Glen Campbell',
  'song_name': 'Gentle On My Mind - Remastered 2001'},
 {'song_first_artist': 'Johnny Cash',
  'song_name': 'Jackson (with June Carter Cash)'},
 {'song_first_artist': 'Buck Owens', 'song_name': 'Act Naturally'},
 {'song_first_artist': 'Floyd Cramer', 'song_name': 'Last Date - Instrumental'},
 {'song_first_artist': 'Dave Dudley', 'song_name': 'Six Days On The Road'},
 {'song_first_artist': 'Bobby Bare', 'song_name': 'Detroit City'},
 {'song_first_artist': 'Merle Haggard', 'song_name': 'Mama Tried'},
 {'song_first_artist': 'Buck Owens', 'song_name': 'Together Again'},
 {'song_first_artist': 'Johnny Cash', 'song_name': 'Daddy Sang

  'song_name': "Once in Royal David's City"},
 {'song_first_artist': 'Royal Philharmonic Orchestra',
  'song_name': 'Away in a Manger'},
 {'song_first_artist': 'Johann Peter Emilius Hartmann',
  'song_name': 'Julebudet til dem, der bygge'},
 {'song_first_artist': 'Gerald Custer', 'song_name': 'Balulalow'},
 {'song_first_artist': 'Traditional', 'song_name': 'In dulci jubilo'},
 {'song_first_artist': 'Ben Parry',
  'song_name': 'Jingle Bells (Arr. B. Parry)'},
 {'song_first_artist': 'Bob Chilcott', 'song_name': "The Shepherd's Carol"},
 {'song_first_artist': 'John Tavener', 'song_name': 'The Lamb'},
 {'song_first_artist': 'John Henry Hopkins Jr.',
  'song_name': 'We Three Kings of Orient Are (arr. J. Rutter): Three kings of '
               'Orient'},
 {'song_first_artist': 'Mykola Dmytrovych Leontovych',
  'song_name': 'Carol of the Bells'},
 {'song_first_artist': 'Traditional', 'song_name': 'The holly and the ivy'},
 {'song_first_artist': 'Anonymous', 'song_name': 'Ding Dong! Merrily o

  'song_name': 'Llenas Las Cuentas (feat. Oscar Maydon)'},
 {'song_first_artist': 'Los Plebes del Rancho de Ariel Camacho',
  'song_name': 'Por No Perderte Te Perdí'},
 {'song_first_artist': 'Gerardo Ortiz', 'song_name': 'Tranquilito'},
 {'song_first_artist': 'ELDANI', 'song_name': 'Bailarina'},
 {'song_first_artist': 'Yorghaki', 'song_name': 'X SUERTE'},
 {'song_first_artist': 'Thony Wilson', 'song_name': 'Una Noche'},
 {'song_first_artist': 'Maria Blaya', 'song_name': 'Me Da Igual'},
 {'song_first_artist': 'Delić', 'song_name': 'No Es Cierto'},
 {'song_first_artist': 'Yarea', 'song_name': 'desde dentro'},
 {'song_first_artist': 'Martox', 'song_name': 'Gucci'},
 {'song_first_artist': 'Ev', 'song_name': 'Levantar'},
 {'song_first_artist': 'Paty B', 'song_name': 'Grande'},
 {'song_first_artist': 'ROLITA', 'song_name': 'Te Quiero Dar'},
 {'song_first_artist': 'Tórtel', 'song_name': 'Algunos de Nosotros'},
 {'song_first_artist': 'Valu', 'song_name': 'Malo'},
 {'song_first_artist': 'Ire', 

 {'song_first_artist': 'Necrophobic',
  'song_name': 'The Infernal Depths of Eternity'},
 {'song_first_artist': 'Lord Of The Lost',
  'song_name': 'For They Know Not What They Do'},
 {'song_first_artist': 'Nightfall', 'song_name': 'Giants of Anger'},
 {'song_first_artist': 'Hanging Garden', 'song_name': 'Nowhere Haven'},
 {'song_first_artist': 'Skepticism', 'song_name': 'Calla'},
 {'song_first_artist': 'Wizardthrone',
  'song_name': 'Forbidden Equations Deep Within the Epimethean Wasteland'},
 {'song_first_artist': 'Insomnium', 'song_name': 'The Reticent'},
 {'song_first_artist': 'Enslaved', 'song_name': 'Fenris - Cinematic Tour 2020'},
 {'song_first_artist': "Tomorrow's Rain",
  'song_name': 'In the Corner of a Dead End Street'},
 {'song_first_artist': 'Zeal & Ardor', 'song_name': 'Run'},
 {'song_first_artist': 'Tribulation', 'song_name': 'Funeral Pyre'},
 {'song_first_artist': 'Sojourner', 'song_name': 'Relics of the Natural Realm'},
 {'song_first_artist': 'King of Asgard', 'song_nam

  'song_name': 'Just the Two of Us (feat. Bill Withers)'},
 {'song_first_artist': 'Bryan Adams', 'song_name': 'Heaven'},
 {'song_first_artist': 'Glenn Frey', 'song_name': "Lover's Moon"},
 {'song_first_artist': 'Paul Young',
  'song_name': 'Every Time You Go Away - Radio Edit'},
 {'song_first_artist': 'Bette Midler', 'song_name': 'Wind Beneath My Wings'},
 {'song_first_artist': 'Air Supply',
  'song_name': 'Making Love Out of Nothing at All'},
 {'song_first_artist': 'Climie Fisher',
  'song_name': 'Love Changes (Everything)'},
 {'song_first_artist': 'Eric Carmen',
  'song_name': 'Hungry Eyes - From "Dirty Dancing" Soundtrack'},
 {'song_first_artist': 'Kim Carnes', 'song_name': 'Bette Davis Eyes'},
 {'song_first_artist': 'Cutting Crew',
  'song_name': '(I Just) Died In Your Arms'},
 {'song_first_artist': 'Chicago',
  'song_name': "Hard to Say I'm Sorry - 2007 Remaster"},
 {'song_first_artist': 'Air Supply', 'song_name': 'All Out of Love'},
 {'song_first_artist': 'The Cars', 'song_name':

  'song_name': 'Uplifting Mantra Pt One'},
 {'song_first_artist': 'LunaRexx ASMR', 'song_name': 'Everything is Okay'},
 {'song_first_artist': 'RoseASMR',
  'song_name': 'Super sleepy attention, close positive whispers, inaudible '
               'whispers Pt.1'},
 {'song_first_artist': 'Summer ASMR', 'song_name': 'Into Dreams'},
 {'song_first_artist': 'DianaDewAsmr', 'song_name': 'Reassuring Whispers Asmr'},
 {'song_first_artist': 'HeatheredEffect ASMR',
  'song_name': 'Close Ear to Ear Whispered Countdown to Sleep Pt.4'},
 {'song_first_artist': 'ASMR Whisperers',
  'song_name': 'Red Candle Whisper Description'},
 {'song_first_artist': 'Mr. Jones ASMR',
  'song_name': 'Unintelligble Sleep Whisper Sayy Wayy'},
 {'song_first_artist': 'LunaRexx ASMR',
  'song_name': 'Removing Negative Energy'},
 {'song_first_artist': 'ASMR Cherry Crush',
  'song_name': "It's Just You and Me"},
 {'song_first_artist': 'Julia Park', 'song_name': "Full Night's Sleep"},
 {'song_first_artist': 'Summer ASMR', 's

 {'song_first_artist': 'Oskar Merikanto',
  'song_name': 'Kesailta (Summer evening), Op. 1'},
 {'song_first_artist': 'Gustav Mahler',
  'song_name': 'Symphony No. 3 in D Minor / Part 2: 3. Comodo. Scherzando. '
               'Ohne Hast (Senza fretta) - Live'},
 {'song_first_artist': 'Göran Söllscher',
  'song_name': '4 Estaciones Portenas (pour guitare): N°.2 Verano Porteño'},
 {'song_first_artist': 'Hugo Alfvén',
  'song_name': 'Midsommarvaka (Midsummer Vigil), Op. 19, "Swedish Rhapsody '
               'No.1"'},
 {'song_first_artist': 'Ludwig van Beethoven',
  'song_name': 'Symphony No. 6 in F Major, Op. 68, S. 464 "Pastoral" (Piano '
               'Transcription by Franz Liszt): V. Hirtengesang. Frohe und '
               'dankbare Gefühle nach dem Sturm. Allegretto'},
 {'song_first_artist': 'Frederick Delius',
  'song_name': 'Delius: 2 Pieces for Small Orchestra: No. 2, Summer Night on '
               'the River'},
 {'song_first_artist': 'Dmitry Kabalevsky',
  'song_name': '6 Pr

In [14]:
# sanity check to see if the shuffling of the library worked

random.shuffle(songs_information)

In [15]:
############### GENIUS ###############

In [16]:
# this part is for the genius info extraction
import re 
!pip install lyricsgenius

import lyricsgenius as lg

In [17]:
# define a function used to get the song id of 
def get_song_lyrics_genius(lg_api, song_name, artist_name):
    
    if artist_name == None:
        song = lg_api.search_song(song_name)
    else:
        song = lg_api.search_song(song_name, artist_name)
    
    if song != None:
        return song.lyrics
    else:
        return None

# define a function to get rid of the brackets in the lyrics objects
def remove_brackets(lyrics):
    return re.sub("\[.*?\]", "", lyrics)

# define a function that puts the string in the given format
def format_lyrics(lyrics):
    
    formatted_lyrics = ""
    
    #remove the brackets
    cleaned_of_brackets = remove_brackets(lyrics)
    
    #remove the empty lines
    split_by_line_break_list = cleaned_of_brackets.split("\n")
    removed_empty_from_list = [lyric_line for lyric_line in split_by_line_break_list if lyric_line.strip() != '']
    
    # combine the lines with the <eos> in the middle
    for lyric_line in removed_empty_from_list:
        formatted_lyrics = formatted_lyrics + lyric_line + " <eos> "
    
    return formatted_lyrics

#validify if a lyric is acutally a lyric
def validify_lyrics(lyrics):
    # check to see if there even are lyrics in the first place
    if lyrics == None:
        return False
    
    #check to see if the count of the nujber of dashes exceeds 10, if it does it means it's a list
    if lyrics.count("-") >= 10:
        return False
    
    # check to see if the length of the string itself 
    if len(lyrics.split()) >= 500:
        return False
    
    return True

In [18]:
# Perry's client access token
client_access_token_genius = "8T4svd_OFcjfLLhc8bB6KEEUrNRExOHrryMfJ7sGzKXK1_ekg_LLdZR7sAr9RX72"

#setting the requests up by passing in the token to genius
lg_api = lg.Genius(client_access_token_genius)


In [19]:
# define a counter to see how many songs we successfully acheived lyrics for
n_succ_in_practice = 0
# copy the original song information list
songs_information_full = []

for i in range(len(songs_information)):
    song_info = songs_information[i]
    lyrics_string = get_song_lyrics_genius(lg_api, song_info["song_name"], song_info["song_first_artist"])
    #lyrics_string = get_song_lyrics_genius(lg_api, song_info["song_name"],None)
    
    if validify_lyrics(lyrics_string) == True:
        # add the song information to the full list by first copying it
        song_info_new = song_info.copy()
        song_info_new["lyrics"] = format_lyrics(lyrics_string)
        songs_information_full.append(song_info_new)
        
        #incrememnt the number of successful queries
        n_succ_in_practice = n_succ_in_practice + 1
        
        # if the number of successful queries is a multiple of 100, write to txt.
        if n_succ_in_practice % 100 == 0:
            
            cleaned_dataset = open(r"cleaned_dataset.txt","w+")
            
            for song_information in songs_information_full:
                cleaned_dataset.write(song_information["lyrics"]+"\n")
            
            cleaned_dataset.close()
                
            print("\n########################\n",n_succ_in_practice," songs successfully added","\n########################\n")
            
            time.sleep(150)

Searching for "Rainbow - Instrumental" by Allen & Bright...
Done.
Searching for "Angels We Have Heard on High - Instrumental" by Chris Rice...
Done.
Searching for "Always" by Waze & Odyssey...
Done.
Searching for "How Far I'll Go - From "Moana"" by Royal Philharmonic Orchestra...
No results found for: 'How Far I'll Go - From "Moana" Royal Philharmonic Orchestra'
Searching for "Oye Como Va" by Tito Puente...
Done.
Searching for "Mary's Boy Child" by Danny Silberman...
Done.
Searching for "Neyð" by Janus Rasmussen...
No results found for: 'Neyð Janus Rasmussen'
Searching for "Mountain at My Gates" by Foals...
Done.
Searching for "Lagrimas Negras" by Bebo Valdés...
Done.
Searching for "Hvíld" by Hugi Guðmundsson...
No results found for: 'Hvíld Hugi Guðmundsson'
Searching for "What If I Never Get Over You" by Lady A...
Done.
Searching for "Where Did Our Love Go" by The Supremes...
Done.
Searching for "Night Visions" by Thirsty Ear...


Timeout: Request timed out:
HTTPSConnectionPool(host='genius.com', port=443): Read timed out. (read timeout=5)

In [ ]:
# sanity check
pprint.pprint(songs_information_full)